# Import libraries and load the Data from the CSV-File


In [1]:
# import everything we'll need
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from prophet import Prophet

# read in the csv-file containing the data
data = pd.read_csv("/Users/erikfriedrich/Downloads/Nat_Gas.csv")

# Fill the gaps in our Data

In [ ]:
# format the date
data['Dates'] = pd.to_datetime(data['Dates'], format='%m/%f/%y')
data['Dates'] = data['Dates'].dt.date # cuts hours, minutes, seconds etc. from the timestamp

# set Date as the index
data = data.set_index('Dates')

In [ ]:
# find beginning and end of our timeframe
start_date = data.index.min()
end_date = data.index.max()

# get all the dates between start and end to assign them prices later
daily_range = pd.date_range(start=start_date, end=end_date, freq="D")
daily_data = pd.DataFrame(index=daily_range)

In [ ]:
# merge data and daily_data
merged_df = daily_data.merge(data, how="left", left_index=True, right_index=True)

# there are only a couple values in our dataframe (at the end of each month)
# to fill the gaps, we use interpolate and approximate the values in a month using a second degree polynomial
merged_df["Prices"] = merged_df['Prices'].interpolate(method='linear',  limit_direction='both')

# we reset the index, so that the date becomes a column again, which is necessary for Prophet
merged_df = merged_df.reset_index()

# rename the columns to ds (referring to the Dates) and y (referring to the price), so that Prophet can work with it
merged_df.columns = ["ds", "y"]

# Use Prophet to predict the values for the next year

In [ ]:
m = Prophet()
m.fit(merged_df)
future = m.make_future_dataframe(periods=365)
prediction = m.predict(future)
m.plot(prediction)

plt.title("Prediction of Natural Gas Prices 365 into the future")
plt.xlabel("Date")
plt.ylabel("Nat Gas Price")
plt.show()
print(prediction)

In [ ]:
# save the relavant dataframes, so that we don't have to run the model, everytime we want to get the approximative price for a given date
prediction.to_csv("/Users/erikfriedrich/Downloads/Nat_Gas_Hat.csv")
merged_df.to_csv("/Users/erikfriedrich/Downloads/Nat_Gas_Filled.csv")

# Write a Function that returns the Price given a Year, a Month and a Day

In [6]:
# load up the files again
df_hat = pd.read_csv("/Users/erikfriedrich/Downloads/Nat_Gas_Hat.csv")
df_fill = pd.read_csv("/Users/erikfriedrich/Downloads/Nat_Gas_Filled.csv")L

In [ ]:
def DateToPrice(year, month, day):
    
    # turns year, month and day into datetime format
    date_obj = datetime(year, month, day)
    formatted_date = date_obj.strftime("%Y-%m-%d")
    datetime_obj = datetime.strptime(formatted_date, "%Y-%m-%d")
    
    # specifies the cutoff date
    cutoff_date = datetime.strptime("2024-09-01", "%Y-%m-%d")
    
    # if the inputted date is greater than the cutoff date, we look into the dataframe with the predicted values
    # otherwise we use the dataframe with the values that we've filled in the beginning
    if date_obj > cutoff_date:
        return df_hat["yhat"].loc[df_hat["ds"] == formatted_date]
    else:
        return df_fill["y"].loc[df_fill["ds"] == formatted_date]

In [ ]:
# Example inputs:

year = 2024
month = 10
day = 30
DateToPrice(year, month, day)